In [ ]:
import sys
sys.path.insert(0, '../')
from tynt import FilterGenerator

In [ ]:
f = FilterGenerator()

In [ ]:
links = f.download_all_links()
tables = f.download_all_tables(links)
bintable = f.fft_table(tables)

In [ ]:
bintable.writeto('../tynt/data/fft.fits', overwrite=True)